In [3]:
import os
from openai import OpenAI
import os
import base64
from dotenv import load_dotenv

load_dotenv()
XAI_API_KEY = os.getenv("X_API_KEY")

image_path = "images/f8532014a537d15a882248dc98d9a0eb.PNG"

client = OpenAI(
    api_key=XAI_API_KEY,
    base_url="https://api.x.ai/v1",
)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")
    return encoded_string

# Getting the base64 string
base64_image = encode_image(image_path)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                    "detail": "high",
                },
            },
            {
                "type": "text",
                "text": "店名有：芋小鱼潮汕糖水（北外滩来福士店）、芋小鱼潮汕糖水（合生汇店）请提取店名、下单时间、价格，并用英语生成json格式，时间数据转化成mongodb时间格式",
            },
        ],
    },
]

completion = client.chat.completions.create(
    model="grok-2-vision-latest",
    messages=messages,
    temperature=0.01,
)

print(completion.choices[0].message.content)

Here is the extracted information in JSON format with the time converted to MongoDB's ISODate format:

```json
[
  {
    "store_name": "芋小鱼潮汕糖水（北外滩来福士店）",
    "order_time": {"$date": "2025-03-09T09:08:00.000Z"},
    "price": "507.08"
  },
  {
    "store_name": "芋小鱼潮汕糖水（合生汇店）",
    "order_time": {"$date": "2025-03-09T10:44:00.000Z"},
    "price": "7.30"
  },
  {
    "store_name": "芋小鱼潮汕糖水（合生汇店）",
    "order_time": {"$date": "2025-03-09T10:43:00.000Z"},
    "price": "220.00"
  }
]
```

Note: The time has been converted to UTC format as required by MongoDB's ISODate. Make sure to adjust the time zone if necessary.
